# Load Data

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

# JSON -> Dataframe

In [2]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
train_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
test_df = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

train_df

,image_id,image_dir,category,genus
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,1
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,1
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,1
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,1
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,1
...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584


# Mapping genus and family

In [3]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
train_df['genus'] = train_df['genus'].map(genus_map)

##Create a family column in the datagframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
train_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in train_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        train_df.at[i, 'family'] = family

train_df

,image_id,image_dir,category,genus,family
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae


# Filtering to Poaceae

In [4]:
#Filter only the images of plants that are in the Poaceae family
train_df = train_df.loc[train_df['family'] == 'Poaceae']
#Reset index
train_df = train_df.reset_index(drop=True)
train_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


# Adding species column

In [5]:
"""#Add category_id and species column
train_df["species"] = None

# Extract category_id and species values from categories where the family is Poaceae
species_list = []
for category in train_meta["categories"]:
    if category["family"] == "Poaceae":
        species_list.append({
            "category": category["category_id"],
            "species": category["species"]
        })

# loop through data frame and species list to update species column
for i, row in train_df.iterrows():
    for species in species_list:
        if row['category'] == species['category']:
            train_df.at[i, 'species'] = species['species']"""

'#Add category_id and species column\ntrain_df["species"] = None\n\n# Extract category_id and species values from categories where the family is Poaceae\nspecies_list = []\nfor category in train_meta["categories"]:\n    if category["family"] == "Poaceae":\n        species_list.append({\n            "category": category["category_id"],\n            "species": category["species"]\n        })\n\n# loop through data frame and species list to update species column\nfor i, row in train_df.iterrows():\n    for species in species_list:\n        if row[\'category\'] == species[\'category\']:\n            train_df.at[i, \'species\'] = species[\'species\']'

# Data visualization

In [6]:
"""
genus_data = train_df['genus'].value_counts().head(15)
genus_data = pd.DataFrame({'Genus' : genus_data.index,
                     'values' : genus_data.values})
                     
plt.figure(figsize = (20, 10))
sns.barplot(x='values', y = 'Genus', data = genus_data , palette='summer_r')
plt.show()

From most to least: Muhlenbergia, Paspalum, Poa, Dichanthelium, Sporobolus, Eragrostis etc.
"""

"\ngenus_data = train_df['genus'].value_counts().head(15)\ngenus_data = pd.DataFrame({'Genus' : genus_data.index,\n                     'values' : genus_data.values})\n                     \nplt.figure(figsize = (20, 10))\nsns.barplot(x='values', y = 'Genus', data = genus_data , palette='summer_r')\nplt.show()\n\nFrom most to least: Muhlenbergia, Paspalum, Poa, Dichanthelium, Sporobolus, Eragrostis etc.\n"

# Filter to two genus

In [7]:
#Muhlenbergia data
muh_pas_df = train_df[(train_df['genus'] == 'Paspalum') | (train_df['genus'] == 'Muhlenbergia')]
muh_pas_df = muh_pas_df.reset_index(drop=True)
muh_pas_df

,image_id,image_dir,category,genus,family
0,09492__001,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
1,09492__003,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
2,09492__004,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
3,09492__005,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
4,09492__006,../input/herbarium-2022-fgvc9/train_images/094...,9492,Muhlenbergia,Poaceae
...,...,...,...,...,...
7347,10398__026,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7348,10398__029,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7349,10398__030,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae
7350,10398__031,../input/herbarium-2022-fgvc9/train_images/103...,10398,Paspalum,Poaceae


# Image displaying

In [8]:
def show_images(genus):
    images = train_df.loc[train_df['genus'] == genus]['image_dir'][:9]
    i = 1
    fig = plt.figure(figsize = (18, 18))
    plt.suptitle(genus, fontsize = '30')
    for image in images:
        img = cv2.imread(image)
        ax = fig.add_subplot(3, 3, i)
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_axis_off()
        i += 1
    plt.show()

In [9]:
#show_images("Muhlenbergia")

# Splitting the dataset into training and validation

**2 Genuses**

In [10]:
muh_df = muh_pas_df[muh_pas_df["genus"] == "Muhlenbergia"] 
pas_df = muh_pas_df[muh_pas_df["genus"] == "Paspalum"]

#15 percent of images will be used for validation
# Muh total: 4228 --> 15% = 634
# Pas total: 3124 --> 15% = 467
muh_valid = muh_df.sample(n=634, random_state=42)
muh_train = muh_df.drop(muh_valid.index)
muh_valid = muh_valid.reset_index(drop=True)
muh_train = muh_train.reset_index(drop=True)

pas_valid = pas_df.sample(n=467, random_state=42)
pas_train = pas_df.drop(pas_valid.index)
pas_valid = pas_valid.reset_index(drop=True)
pas_train = pas_train.reset_index(drop=True)

# Merging the Muhlanbergia and Paspalum databases
muh_pas_train = pd.concat([muh_train, pas_train])
muh_pas_train = muh_pas_train.reset_index(drop=True)

muh_pas_valid = pd.concat([muh_valid, pas_valid])
muh_pas_valid = muh_pas_valid.reset_index(drop=True)

muh_pas_train["genus"].nunique()

2

In [11]:
train_df["genus"].value_counts().tail(20)

Otatea             33
Chaetium           30
Trichoneura        30
Coleanthus         28
Tetrapogon         27
Arctopoa           21
Willkommia         20
Setariopsis        20
Tuctoria           19
Neostapfia         18
Chusquea           18
Amelichloa         16
Allolepis          14
Ptilagrostis       14
Swallenia          14
Ptilagrostiella    14
Rhipidocladum      11
Dupontia           10
Kalinia            10
Barkworthia         8
Name: genus, dtype: int64

# Creating the model

In [12]:
import os
import torch
import torchvision
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn import preprocessing
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.models import resnet50, ResNet50_Weights
!pip install fvcore

df = train_df

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=def642efa2746112ae02a8ec8335e95e4c5399a0d5fa926a325e2bc1e716bad8
  Stored in directory: /root/.cache/pip/wheels/af/cd/23/3fb62ec8606cb08cc18abb8d67bec255baf353623be889da1e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=db71b61562880efabcbf53f69615731978720effcfe2962d7cc28257a536cb9a
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath


In [13]:
# Split the dataset for each class separately
train_dfs = []
val_dfs = []
for label in df['genus'].unique():
    # Filter the dataset to only include images with the current label
    label_df = df[df['genus'] == label]
    
    # Split the dataset into training and evaluation sets
    train_df, val_df = train_test_split(label_df, test_size=0.2)
    
    # Append the training and evaluation sets to their respective lists
    train_dfs.append(train_df)
    val_dfs.append(val_df)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)

train_df['genus'] = pd.factorize(train_df['genus'])[0]
val_df['genus'] = pd.factorize(val_df['genus'])[0]

In [14]:
batch_size = 128
epochs = 10
IM_SIZE = 256

X_train, Y_train = train_df["image_dir"].values, train_df["genus"].values

X_val, Y_val = val_df["image_dir"].values, val_df["genus"].values

Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [15]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        elif "test" in self.fnames[index]:            
            return self.transform(x), self.fnames[index]

                
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

N_Classes = train_df['genus'].nunique()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest101', pretrained=True)

/opt/conda/lib/python3.7/site-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/zhanghang1989/ResNeSt/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/zhanghang1989/ResNeSt/releases/download/weights_step1/resnest101-22405ba7.pth" to /root/.cache/torch/hub/checkpoints/resnest1

  0%|          | 0.00/185M [00:00<?, ?B/s]

In [16]:
"""
# LeNet Model definition
class Net(nn.Module):
    def __init__(self, num_classes=N_Classes, input_channels=3, input_size=IM_SIZE):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        
        # Calculate the dimensions after the convolutions and max pooling
        dim_after_conv1 = (input_size - 4) // 2  # First convolution and max pooling
        dim_after_conv2 = (dim_after_conv1 - 4) // 2  # Second convolution and max pooling
        
        self.fc1_input_size = 20 * dim_after_conv2 * dim_after_conv2
        self.fc1 = nn.Linear(self.fc1_input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.fc1_input_size)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net(num_classes=N_Classes, input_channels=3, input_size=IM_SIZE).to(device)"""

'\n# LeNet Model definition\nclass Net(nn.Module):\n    def __init__(self, num_classes=N_Classes, input_channels=3, input_size=IM_SIZE):\n        super(Net, self).__init__()\n        self.conv1 = nn.Conv2d(input_channels, 10, kernel_size=5)\n        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)\n        self.conv2_drop = nn.Dropout2d()\n        \n        # Calculate the dimensions after the convolutions and max pooling\n        dim_after_conv1 = (input_size - 4) // 2  # First convolution and max pooling\n        dim_after_conv2 = (dim_after_conv1 - 4) // 2  # Second convolution and max pooling\n        \n        self.fc1_input_size = 20 * dim_after_conv2 * dim_after_conv2\n        self.fc1 = nn.Linear(self.fc1_input_size, 50)\n        self.fc2 = nn.Linear(50, num_classes)\n\n    def forward(self, x):\n        x = F.relu(F.max_pool2d(self.conv1(x), 2))\n        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))\n        x = x.view(-1, self.fc1_input_size)\n        x = F.relu(se

In [17]:
print(model.fc.in_features) 
print(model.fc.out_features)

for param in model.parameters():
    param.requires_grad= False
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, N_Classes)
model.fc = last_layer
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters()) 

2048
1000
158


In [18]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average=None)
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1               

In [19]:
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average=None)
    return eval_acc/len(testloader), eval_loss/len(testloader), eval_f1    

In [20]:
scaler = torch.cuda.amp.GradScaler(enabled=True)
for epoch in range(epochs):
    train_acc, train_loss, train_f1 = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1 = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1: {train_f1}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1: {eval_f1}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):")
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)


100%|██████████| 85/85 [06:30<00:00,  4.59s/it]


Epoch 1 | Train Acc: 23.02658462524414 | Train Loss: 3.364929676055908 | Train F1: [0.19104478 0.         0.02371542 0.         0.         0.27739464
 0.09210526 0.         0.         0.         0.33570863 0.14545455
 0.         0.         0.00888889 0.         0.         0.21311475
 0.19047619 0.26375712 0.10884354 0.29606049 0.07398568 0.
 0.03838772 0.         0.         0.02898551 0.13454545 0.
 0.046875   0.         0.03053435 0.15       0.         0.01754386
 0.05073996 0.         0.18348624 0.41109795 0.05430933 0.
 0.         0.         0.         0.11371237 0.         0.
 0.13384813 0.         0.26872499 0.03278689 0.         0.42857143
 0.10687023 0.15231788 0.11898323 0.25560538 0.         0.28363636
 0.         0.         0.         0.05882353 0.03703704 0.
 0.         0.00869565 0.         0.         0.08108108 0.
 0.         0.         0.         0.         0.30839002 0.03761755
 0.05643739 0.09756098 0.         0.         0.01694915 0.12738854
 0.         0.         0.  

 43%|████▎     | 143/335 [11:15<15:06,  4.72s/it]


KeyboardInterrupt: 